<a href="https://colab.research.google.com/github/ort-eila/csiseminar/blob/main/project/ste3_vit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# ! pip install datasets transformers
# !pip install --upgrade datasets


In [13]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
import os
rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_RGB_224_224/train_validation_test"


# debug_rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/output_folder"
output_json_file = os.path.join(rgb_images_path,'metadata.json')   # Output metadata JSON file name
output_json_file

'/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_RGB_224_224/train_validation_test/metadata.json'

In [15]:
import datasets

print("datasets version:", datasets.__version__)


datasets version: 2.14.4


In [16]:
import os
os.listdir(rgb_images_path)

['train', 'validation', 'test']

In [17]:
os.path.basename(rgb_images_path)

'train_validation_test'

In [20]:
import os
import json

# Define the root folder
root_folder = rgb_images_path  # Replace with your root folder containing subfolders
output_json_file = os.path.join(root_folder,'metadata.json')   # Output metadata JSON file name

metadata = []

# Iterate through subfolders (train, validation, test)
for split_folder in os.listdir(root_folder):
    split_folder_path = os.path.join(root_folder, split_folder)

    if os.path.isdir(split_folder_path):
        for label in os.listdir(split_folder_path):
            label_folder = os.path.join(split_folder_path, label)

            if os.path.isdir(label_folder):
                for image_file in os.listdir(label_folder):
                    if image_file.endswith('.jpg'):
                        image_path = os.path.join(label_folder, image_file)
                        metadata.append({"image_path": image_path, "label": label, "split": split_folder})

# Save metadata as JSON file
with open(output_json_file, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print("Metadata JSON file created:", output_json_file)


Metadata JSON file created: /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_RGB_224_224/train_validation_test/metadata.json


In [21]:
output_json_file

'/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_RGB_224_224/train_validation_test/metadata.json'

In [22]:
from PIL import Image
from datasets import load_dataset, ClassLabel
# Define the labels
class_labels = ClassLabel(names=["nonprogressor", "progressor"])
# class_labels = ClassLabel(names=["negative", "positive"])


# Load the dataset
ds = load_dataset("json", data_files=output_json_file)
# ,split=['train'])

# Print some information about the dataset
print(ds)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'split', 'image_path'],
        num_rows: 4596
    })
})


In [23]:
# Function to load the image using PIL and add it to the dataset
def load_pil_image(example):
    image_path = example["image_path"]
    pil_image = Image.open(image_path)
    example["pil_image"] = pil_image
    return example

In [ ]:
# Apply the function to each example in the dataset
ds = ds.map(load_pil_image)

Map:   0%|          | 0/4596 [00:00<?, ? examples/s]

In [ ]:
from PIL import Image
import numpy as np
from IPython.display import display


# Access an example and its PIL image
ex = ds['train'][0]
print("Example:", ex)

# Display the PIL image
display(ex["pil_image"])

# Convert PIL image to NumPy array
image_array = np.array(ex["pil_image"])

# Display image array shape and pixel values
print("Image array shape:", image_array.shape)
print("Min pixel value:", np.min(image_array))
print("Max pixel value:", np.max(image_array))

In [ ]:
label = ds['train'][0]['label']
label

In [ ]:
class_labels

In [ ]:
class_labels.str2int(label)

In [ ]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
feature_extractor

In [ ]:
ex = ds['train'][0]
image = ex['pil_image']
feature_extractor(image, return_tensors='pt')

In [ ]:
def process_example(example):
    inputs = feature_extractor(example['pil_image'], return_tensors='pt')
    inputs['label'] = example['label']
    return inputs

In [ ]:
process_example(ds['train'][0])

In [ ]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['pil_image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = example_batch['label']
    return inputs

prepared_ds = ds.with_transform(transform)

In [ ]:
prepared_ds['train'][0:2]

In [ ]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [ ]:
class_labels.names

In [ ]:
from transformers import ViTForImageClassification

labels = class_labels.names #ds['train'].features['labels'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans-demo-v5",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'dcis-eila',
    "tags": ['image-classification'],
}

if training_args.push_to_hub:
    trainer.push_to_hub('🍻 cheers', **kwargs)
else:
    trainer.create_model_card(**kwargs)